In [14]:
#notebook for scraping
#dependencies 

import os 
from bs4 import BeautifulSoup
import requests
from splinter import Browser
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager


In [2]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======


[WDM] - Current google-chrome version is 90.0.4430
[WDM] - Get LATEST driver version for 90.0.4430
[WDM] - Driver [/Users/summyabakshi/.wdm/drivers/chromedriver/mac64/90.0.4430.24/chromedriver] found in cache


In [3]:
#website to scrape 
url = 'https://redplanetscience.com/'
#retrieve page wiht splinter module - this page will need to stay open to retreive the parsed text
browser.visit(url)


In [4]:
html = browser.html
#parse with soup
soup = BeautifulSoup(html, 'html.parser')

#### Scrape the Mars News Site and collect the latest News Title and Paragraph Text. Assign the text to variables that you can reference later.

In [5]:
#parent divs for latest article title and parargaraph

title = soup.find('div', class_='content_title').text
print(title)
para = soup.find('div', class_='article_teaser_body').text
print(para)



NASA's Perseverance Rover Attached to Atlas V Rocket
Ready for its launch later in the month, the Mars-bound rover will touch terra firma no more.


In [6]:
#remeber to quit broswer 
browser.quit()

#### JPL Mars Space Images - Featured Image

In [8]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 90.0.4430
[WDM] - Get LATEST driver version for 90.0.4430


[WDM] - Driver [/Users/summyabakshi/.wdm/drivers/chromedriver/mac64/90.0.4430.24/chromedriver] found in cache


In [9]:
jpl_url = 'https://spaceimages-mars.com/'
#open the website for scraping
browser.visit(jpl_url)



In [10]:
html = browser.html
#parse
soup = BeautifulSoup(html, 'html.parser')


In [11]:
featured_img_url = soup.find('img', class_="headerimage fade-in")['src']
featured_img_url = (jpl_url + featured_img_url)
print(featured_img_url)

https://spaceimages-mars.com/image/featured/mars1.jpg


In [12]:
browser.quit()

### Pandas Scraping 

#### Mars Facts

In [15]:
mars_facts_url = 'https://galaxyfacts-mars.com/'
#create a tablular structure out of the URL
table = pd.read_html(mars_facts_url)
table

[                         0                1                2
 0  Mars - Earth Comparison             Mars            Earth
 1                Diameter:         6,779 km        12,742 km
 2                    Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 3                   Moons:                2                1
 4       Distance from Sun:   227,943,824 km   149,598,262 km
 5          Length of Year:   687 Earth days      365.24 days
 6             Temperature:     -87 to -5 °C      -88 to 58°C,
                       0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:          2 ( Phobos & Deimos )
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC

In [42]:
facts_df = table[0]
facts_df.head()

,0,1,2
0,Mars - Earth Comparison,Mars,Earth
1,Diameter:,"6,779 km","12,742 km"
2,Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
3,Moons:,2,1
4,Distance from Sun:,"227,943,824 km","149,598,262 km"


In [43]:
facts_df = facts_df.rename(columns={0:'Mars-Earth Comparison', 1: 'Mars', 2:'Earth' })
facts_df

,Mars-Earth Comparison,Mars,Earth
0,Mars - Earth Comparison,Mars,Earth
1,Diameter:,"6,779 km","12,742 km"
2,Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
3,Moons:,2,1
4,Distance from Sun:,"227,943,824 km","149,598,262 km"
5,Length of Year:,687 Earth days,365.24 days
6,Temperature:,-87 to -5 °C,-88 to 58°C


In [44]:
facts_df = facts_df.drop([0])


In [45]:
facts_df = facts_df.set_index(['Mars-Earth Comparison'])
facts_df.head()

,Mars,Earth
Mars-Earth Comparison,,
Diameter:,"6,779 km","12,742 km"
Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons:,2,1
Distance from Sun:,"227,943,824 km","149,598,262 km"
Length of Year:,687 Earth days,365.24 days


In [46]:
#convert table to html string
html_table = facts_df.to_html()
html_table


'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars</th>\n      <th>Earth</th>\n    </tr>\n    <tr>\n      <th>Mars-Earth Comparison</th>\n      <th></th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Diameter:</th>\n      <td>6,779 km</td>\n      <td>12,742 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg</td>\n      <td>5.97 × 10^24 kg</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2</td>\n      <td>1</td>\n    </tr>\n    <tr>\n      <th>Distance from Sun:</th>\n      <td>227,943,824 km</td>\n      <td>149,598,262 km</td>\n    </tr>\n    <tr>\n      <th>Length of Year:</th>\n      <td>687 Earth days</td>\n      <td>365.24 days</td>\n    </tr>\n    <tr>\n      <th>Temperature:</th>\n      <td>-87 to -5 °C</td>\n      <td>-88 to 58°C</td>\n    </tr>\n  </tbody>\n</table>'

#### Mars Hemispheres

In [48]:
#hesmisphere website for scraping
img_url = 'https://marshemispheres.com/'
#retreive page with requests module 
response = requests.get(img_url)
#create soup object with parser
soup = BeautifulSoup(response.text, 'html.parser')


In [49]:
print(soup.prettify())

<!DOCTYPE html>
<html lang="en">
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <link href="css/jquery-ui.css" rel="stylesheet" type="text/css"/>
  <title>
   Astropedia Search Results | GUSS Astrogeology Science Center
  </title>
  <meta content="GUSS Astrogeology Science Center Astropedia search results." name="description"/>
  <meta content="GUSS,Astrogeology Science Center,Cartography,Geology,Space,Geological Survey,Mapping" name="keywords"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <meta content="width=device-width, initial-scale=1, maximum-scale=1" name="viewport"/>
  <link href="css/main.css" media="screen" rel="stylesheet"/>
  <link href="css/print.css" media="print" rel="stylesheet"/>
  <link href="#" rel="icon" type="image/x-ico"/>
 </head>
 <body id="results">
  <header>
   <a href="#" style="float:right;margin-top:10px;" target="_blank">
    <img alt="USGS: Science for a Changing World" class="logo" height="60" src="images

In [67]:
results = soup.find_all('a', class_ ='itemLink product-item')
results

[<a class="itemLink product-item" href="cerberus.html"><img alt="Cerberus Hemisphere Enhanced thumbnail" class="thumb" src="images/39d3266553462198bd2fbc4d18fbed17_cerberus_enhanced.tif_thumb.png"/></a>,
 <a class="itemLink product-item" href="cerberus.html">
 <h3>Cerberus Hemisphere Enhanced</h3>
 </a>,
 <a class="itemLink product-item" href="schiaparelli.html"><img alt="Schiaparelli Hemisphere Enhanced thumbnail" class="thumb" src="images/08eac6e22c07fb1fe72223a79252de20_schiaparelli_enhanced.tif_thumb.png"/></a>,
 <a class="itemLink product-item" href="schiaparelli.html">
 <h3>Schiaparelli Hemisphere Enhanced</h3>
 </a>,
 <a class="itemLink product-item" href="syrtis.html"><img alt="Syrtis Major Hemisphere Enhanced thumbnail" class="thumb" src="images/55a0a1e2796313fdeafb17c35925e8ac_syrtis_major_enhanced.tif_thumb.png"/></a>,
 <a class="itemLink product-item" href="syrtis.html">
 <h3>Syrtis Major Hemisphere Enhanced</h3>
 </a>,
 <a class="itemLink product-item" href="valles.html"><

In [70]:
for result in results:
    h_url = result.find('img alt', class_ = 'thumb')['src']
    full_url = (img_url+h_url)
    print(full_url)
    print('---------')

    

TypeError: 'NoneType' object is not subscriptable